In [ ]:
import numpy as np
import matplotlib as plt

In [ ]:
f_runtime = np.array([1,0.768])   # HVAC system runtime fraction
f_recir = np.array([0.438, 0.05]) # Recirculated air fraction
student_class = np.array([24,30])
total_students = np.array([868, 1873])
portion = np.array([[0.033,0.967,0],[0.003,0.709,0.288]])
total_class = np.round(total_students / student_class)
student_common = np.array([664,1873])
student_recir = np.array([592,1843])

# Volume of class propotional to no. of students per class
V_class = 4 * student_class # +/- 25% as a range
V_recir = np.array([13832, 33600]) # +/- 25% as a range
V_common = 1.39 * student_common
V = np.concatenate((V_class.reshape((2,1)),V_recir.reshape((2,1)),V_common.reshape((2,1))),axis = 1)

n_filter = np.array([0.12, 0.99]) 
q_total = np.array([1019.4*60,595*60]) # total HVAC capacity (m3/hour)
q_supply = np.array([28.3*60, 8.5*60])  # supply airflow rate of one classroom (m3/hour)
q_return = q_supply * f_recir # Return air flow rate of HVAC system in classroom
lambda_infilltration = np.array([0.31,0.31]) # natural ventilation rate
K_deposition = np.zeros((2,3)) # deposition rate of measles bio-aerosols
K_deposition.fill(1.7)

infection_period = np.array([3,4])
q = np.array([1925,2765])
total_time = 760

In [ ]:
# probability of infection transmission in a wellmixed indoor space
# P_infection = Number of Infected Cases / Number of Susceptible Individuals
# quanta = -ln(1-P_infection)
def P_infection(quanta):
    return 1 - np.exp(-quanta)
P_infection(1)

In [ ]:
def vaccine_coverage(total_students, vaccine_rate):
    vaccine = np.zeros(vaccine_rate.shape)
    for i in range(vaccine.shape[0]):
        vaccine[i] = np.round(total_students[i] * vaccine_rate[i])
    return vaccine
    
vacc_of_students = vaccine_coverage(total_students,portion)
print(vacc_of_students)

In [ ]:
# calculate number of susceptible based on age & vaccine coverage
Susceptible_rate = np.array([[1,0.1,0.01],[1,0.05,0.05]])
def Susceptible(type, vaccine):
    res = 0
    for i in range(len(vaccine)):
        res += round(Susceptible_rate[type][i] * vaccine[i])
    return res

sus = np.zeros((2))
inflict = np.array([28,69])

for i in range(2):
    sus[i] = Susceptible(i,vacc_of_students[i])

p_inflict = inflict / sus
p_inflict_per_day = p_inflict / infection_period
reverse_mu = -np.log(1-p_inflict_per_day)
print(sus,p_inflict_per_day,reverse_mu)


In [ ]:
# total removal rate of measles viruses in space i
def K_total(l_infiltration, K_deposition, K_ventilation, K_filtration, K_purification):
    y = np.zeros(K_ventilation.T.shape)
    for i in range(y.shape[0]):
        y[i] = l_infiltration
    l_infiltration = y.T
    return l_infiltration + K_deposition + K_ventilation + K_filtration + K_purification

#  mechanical ventilation rate of HVAC systems for space i
def K_ventilation(f_runtime, f_recir, q_return, V):
    return (f_runtime * (1 - f_recir) * q_return / V.T).T

# particle removal rate due HVAC filtration for space i
def K_filtration(f_runtime, f_recir, n_filter, q_return, V):
    return (f_runtime * f_recir * n_filter * q_return / V.T).T

# removal efficiency of measles viruses by an air purifier in space i
def K_purification(f_runtimeAP, CDAR, V):
    return f_runtimeAP * CDAR / V

3 spaces: Infector’s classroom, Recirculation spaces, Common spaces

In [ ]:
K_ventilation_test = K_ventilation(f_runtime, f_recir, q_return, V)
print(K_ventilation_test) 
K_filtration_test = K_filtration(f_runtime, f_recir, n_filter, q_return, V)
print(K_filtration_test)
K_total_test = K_total(lambda_infilltration, K_deposition, K_ventilation_test, K_filtration_test, 0)
print(K_total_test)

In [ ]:
# Concentration of quanta in infector’s classroom and common space t hours 
# after presence of the index case s in space i
def c_quanta(I, q, V, K_total, t):
    return I * q / V / K_total * (1 - np.exp(-K_total*t/60))

def cal_c_quanta_recir(target):
    q_supply_recir = q_supply * (total_class - 1)
    F_class_recir = q_return * f_recir * f_runtime * (1- n_filter) / V_recir * q_supply_recir / q_total
    C_quanta_recir = np.zeros((target,2))
    for t in range(1,target):
        delta_t = 1/60
        K_total_recir = K_total_test[:,1] 
        C_quanta_class = c_quanta(1,q,V_class,K_total_test[:,0],t-1)
        C_quanta_recir[t] = delta_t * (- K_total_recir * C_quanta_recir[t-1] + F_class_recir * C_quanta_class) + C_quanta_recir[t-1]
    return C_quanta_recir
C_quanta_recir = cal_c_quanta_recir(total_time)
print(C_quanta_recir)

In [ ]:
time_class = np.array([280,340])
time_recir = np.array([280,340])
time_common = np.array([20,70]) 
inhale = np.array([12.96/24,15.53/24]) # inhalation rate (m3/hour)
# average number of quanta breathed by susceptible students during a typical school day
def average_quanta(total_students, p, target):
    mu = np.zeros((2))
    # infector classroom
    f_infector = np.zeros((target,2))
    f_recir = np.zeros((target,2))
    f_common = np.zeros((target,2))
    for t in range(target): 
        f_infector[t]  = student_class * c_quanta(1,q,V_class,K_total_test[:,0], t)
        f_common[t] = student_common * c_quanta(1,q,V_common,K_total_test[:,2],t)
        f_recir[t] = student_recir * C_quanta_recir[t]
    for i in range(2):
        for t in range(time_class[i]+1):
            if t == 0 or t == time_class[i]:
                mu[i] += f_infector[t][i]
            else:
                mu[i] += 2*f_infector[t][i]

        for t in range(time_recir[i]+1):
            if t == 0 or t == time_recir[i]:
                mu[i] += f_recir[t][i]
            else:
                mu[i] += 2*f_recir[t][i]
                
        for t in range(time_common[i]+1):
            if t == 0 or t == time_common[i]:
                mu[i] += f_common[t][i]
            else:
                mu[i] += 2*f_common[t][i]
    mu = mu / 30 * p / 60 /total_students
    return mu 

mu = average_quanta(total_students,inhale,total_time)  
print(mu)
print(P_infection(mu))  